In [1]:
import os 
import pandas as pd
import numpy as np
from pydub import AudioSegment

import IPython.display as ipd
import warnings
import librosa
import librosa.display
import librosa.feature
import matplotlib.pyplot as plt
from tqdm import tqdm
import utils

import processing
import event_manager
import model_training
import epoching

warnings.filterwarnings('ignore')

/Users/bastienorset/Documents/git/respiratory_sound/.venv/lib/python3.10/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
audio_folder = '../../data/ICBHI_final_database'
list_audio_files = processing.get_list_recording(audio_folder)
print('Found {} recording files'.format(len(list_audio_files)))

Found 920 recording files


In [3]:
label_list = []
epochs_list = []
n_fft=400
win_length = 0.100
list_series = list()
for audio_file in tqdm(list_audio_files):
    data_file = processing.load_file_from_recording_name(audio_folder,audio_file)
    data_filtered,df_label = processing.preprocess_data(data_file,annotations='event')
    data_feature= processing.apply_stft(data_filtered,n_fft=400,center=False)
    df_label = event_manager.segment_event_annotation(data_filtered,data_feature['time'])
    epochs_,label_ = epoching.get_epoching_from_label(data_feature,df_label,mean=True)
    epochs_list.append(epochs_)
    label_list.append(label_)

100%|██████████| 920/920 [07:16<00:00,  2.11it/s]


In [7]:
df_label

,start,end,label
0,0.000,0.100,0
1,0.025,0.125,0
2,0.050,0.150,0
3,0.075,0.175,0
4,0.100,0.200,0
...,...,...,...
792,19.800,19.900,0
793,19.825,19.925,0
794,19.850,19.950,0
795,19.875,19.975,0


In [4]:
X = model_training.prepare_data_for_modeling(epochs_list,label_list,list_audio_files)

In [5]:
X.label.value_counts()

0    731521
1     45564
2     11049
Name: label, dtype: int64

In [6]:
X.to_feather('../../data/data_model/data_epoching_event.ftr')